In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.8 MB/s eta 0:00:00


In [2]:
from transformers import logging
logging.set_verbosity_error()

In [4]:
import pandas as pd
data = pd.read_csv("/full_train.csv")
print(data['Comment'][0:10])

0    Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trô...
1    Gọi ship 1 xuất cari gà bánh naan và 3 miếng g...
2    Thời tiết lạnh như này, cả nhà rủ nhau đến leg...
3    Em có đọc review thấy mng bảo trà sữa nướng đề...
4    Đồ ăn rất ngon, nhà hàng cũng rất đẹp, tất cả ...
5    Đồ ăn ngon, đặt qua now shop để k cẩn thận bị ...
6    Nhà hàng yêu thích của mình đây nè! Đến đây mì...
7    🔸Vị trí: mặt đường Trung Yên rất dễ tìm, một d...
8    Mình đã ăn tại đây và rất ngon,\nChuẩn vị phố ...
9    Thịt mềm, sốt ngon, dưa góp ăn cùng đỡ ngán\nM...
Name: Comment, dtype: object


In [5]:
data = data.drop('Unnamed: 0', axis = 1)
data = data.drop('RevId', axis = 1)
data = data.drop('UserId', axis = 1)
data = data.drop('image_urls', axis = 1)
data

,Comment,Rating
0,"Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trô...",1.0
1,Gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,0.0
2,"Thời tiết lạnh như này, cả nhà rủ nhau đến leg...",1.0
3,Em có đọc review thấy mng bảo trà sữa nướng đề...,0.0
4,"Đồ ăn rất ngon, nhà hàng cũng rất đẹp, tất cả ...",1.0
...,...,...
9068,Thực sự mà nói thấy mọi người đánh giá nhiều q...,1.0
9069,👉👉👉LẨU THÁI TÔMYUM\n👉👉Đc: 19 Lò Đúc-HBT-HN\n👉T...,1.0
9070,Ngay từ lúc đầu tiên bước vào nhà hàng đã được...,1.0
9071,Đặt ăn thử mà thấy ngón cá...! 🤩 sẽ còn ủn hộ ...,1.0


In [12]:
import torch
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

In [13]:
len(tokenizer.get_vocab())

64001

In [14]:
corpus = []
for tmp in zip(data["Comment"], data["Rating"]):
  sent, lb = tmp

  if(float(lb) == 1):
    corpus.append((sent, 1))
  if float(lb) == 0:
    corpus.append((sent, 0))
print(corpus[0])
print(len(corpus))



('Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trông rất thích', 1)
9070


In [15]:
def segment(corpus):
  ct = []
  lb = []
  for content, label in corpus:
#     if(type(content) == str):
      ct.append(content)
      lb.append(label)
  return ct, lb

In [16]:
sents, labels = segment(corpus)
print(sents[0], " ", labels[0])

Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trông rất thích   1


In [17]:
print(len(sents))

9070


In [18]:
import numpy as np

In [19]:
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed_everything(86)

In [20]:
k = 0
for sent in sents:
  if(sent.find("bình luận không xác định") != -1):
    print(sent)
    k+=1
k

0

In [21]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
def check_spell(string):
  for c in string :
    if c in s1:

      return True
  return False

checked_sents = []

for sent in sents:
  if(not check_spell(sent)):
    checked_sents.append(sent)

print(len(checked_sents))

86


In [ ]:
for i in range(10):
  print(i, checked_sents[i])

In [23]:
from sklearn.model_selection import train_test_split

train_sents, valid_sents, train_labels, valid_labels = train_test_split(sents, labels, test_size=0.2)
print(len(train_sents), " ", len(valid_sents))

7256   1814


In [24]:
len(train_sents)

7256

In [25]:
sum_1 = 0
sum_2 = 0
for label in train_labels:
  if label == 0:
    sum_1 += 1
  else:
    sum_2 += 1
print(sum_1, " ", sum_2)
sum_1 = 0
sum_2 = 0
for label in valid_labels:
  if label == 0:
    sum_1 += 1
  else:
    sum_2 += 1
print(sum_1, sum_2)

1539   5717
385 1429


In [26]:
print(train_sents[0])
print( train_labels[0])

Trước giờ mình nghiện nướng lụi Ngô Thì Nhậm mà quán lại đóng cửa huhu 😭😭. Vừa may tình cờ tìm được quán nướng 69 mới mở gần trường vào ăn thử rồi lại nghiện luôn nên review chia sẻ cho các bạn.
🛣 Địa điểm: Quán ở đối diện với chè bưởi Tạ Quang Bửu, chếch cây xăng. Khu này khá nhiều quán bia và lẩu nướng. Khách ăn có thể để xe ngay trước quán hoặc bên cạnh, có nhân viên trông xe.
❤️ Đồ ăn: mình ấn tượng nhất ở quán là nước chấm sốt me, đặc sánh, chua ngọt vị me chứ ko pha loãng như các quán nướng lụi khác. Thịt ướp đậm đà và sạch sẽ nữa vì nhìn thấy trực tiếp anh chủ quán làm luôn! Có lẽ mình thích nhất ở đây là nầm nướng, nầm ăn giòn và tươi, nhóm mình gọi 3 đĩa nầm liền luôn vì ngon quá 😂😂. Quán có cả lẩu Thái khá đậm vị Thái, có cả ngao và mực. Mình đi ăn 6 người, ăn nướng no bét nhè rồi nên ăn lẩu 1 nồi no căng luôn.
❤️ Giá: Ăn bét nhè bàn 6 người mà đứng dậy shock khi thanh toán hết có 500k 😂😂😂. Mỗi đĩa thịt đồng giá 50k mà đầy muốn rơi ra ngoài, ko hề độn gì luôn ạ 😅.
> Nhất định

In [ ]:
pip install Keras-Preprocessing

In [29]:
import re
from keras_preprocessing.sequence import pad_sequences
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from gensim.utils import simple_preprocess

In [ ]:
pip install underthesea

In [31]:
from underthesea import text_normalize

In [33]:
def purify(text):
  vietnamese_characters = "a-zA-Z0-9ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂẠẢẤẦẨẪẬẮẰẲẴẶẸẺẼỀỀỂẾưăạảấầẩẫậắằẳẵặẹẻẽềềểếỄỆỈỊỌỎỐỒỔỖỘỚỜỞỠỢỤỦỨỪễệỉịọỏốồổỗộớờởỡợụủứừỬỮỰỲỴÝỶỸửữựỳýỵỷỹ"
  text = text.lower()
  text = text.strip()
  # text = " "+ text + " "
  text = re.sub("[+]", ' ', text) #all math-operating characters removed
  text = re.sub(" \\[a-z]", ' ', text)# all newline and slash characters removed

  text = re.sub('[^' + vietnamese_characters + ']+', ' ', text) # all special characters removed
  text = re.sub("([A-Za-z]+[0-9]+)|([0-9]+[A-Za-z]+)", " ", text) #150g
  text = re.sub("[0-9]{3,}", " ", text) #all too long numbers (>= 3 digits) removed
  text = re.sub(r'(\D)\1{2,}', r'\1', text) # all consecutive duplicated characters removed
  text = text.strip()

  #all single-charater words removed
  tmp_text = []
  for x in text.split(" "):
    if(len(x) > 1):
      tmp_text.append(x)

  text = " ".join(tmp_text)
  text = text_normalize(text)
  return text


In [35]:
from underthesea import word_tokenize

In [37]:
def preprocess(purified_sents, max_len=None):
  preprocessed_sents = []
  for purified_text in purified_sents:

#     preprocessed by word_tokenize of underthesea
    text = word_tokenize(purified_text, format = "text")
    tmp_text = text

    if(max_len is not None):
      words = tmp_text.split(" ")
      if(len(words) > max_len):
        words = words[0:max_len]
      tmp_text = " ".join(words)

    preprocessed_sents.append(tmp_text)
  return preprocessed_sents


In [38]:
def encode_plus(preprocessed_sents,max_len=256):
  encoded_sents = []
  masks = []
  for sent in preprocessed_sents:
    sent_info = tokenizer.encode_plus(sent,
                                      padding='max_length',
                                      truncation=True,
                                      add_special_tokens=True,
                                      max_length = max_len,
                                      return_token_type_ids=False,
                                      return_attention_mask=True,
                                     return_tensors='np')
    encoded_sent = sent_info['input_ids'].flatten()
    mask =  sent_info['attention_mask'].flatten()
    encoded_sents.append(encoded_sent)
    masks.append(mask)
  return encoded_sents, masks

In [39]:
train_purified_sents = [purify(train_sent) for train_sent in train_sents]
valid_purified_sents = [purify(valid_sent) for valid_sent in valid_sents]

In [40]:
train_preprocessed_sents = preprocess(train_purified_sents)
valid_preprocessed_sents = preprocess(valid_purified_sents)

In [41]:
train_ids, train_masks = encode_plus(train_preprocessed_sents, max_len=256)
valid_ids, valid_masks = encode_plus(valid_preprocessed_sents, max_len=256)

In [42]:
import numpy as np

In [43]:
batch_size = 32
train_inputs = torch.tensor(train_ids)
valid_inputs = torch.tensor(valid_ids)
train_labels = torch.tensor(train_labels)
valid_labels = torch.tensor(valid_labels)
train_masks = torch.tensor(train_masks)
valid_masks = torch.tensor(valid_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

valid_data = TensorDataset(valid_inputs, valid_masks, valid_labels)
valid_sampler = RandomSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

<ipython-input-43-b0722a78d24c>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_inputs = torch.tensor(train_ids)


In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [45]:
device

device(type='cuda')

In [46]:
import torch.nn as nn
import torch.nn.functional as F

In [48]:
class BERT_for_sentiment_large(nn.Module):
  def __init__(self, output_size=2, dropout=0.1):
    super(BERT_for_sentiment_large, self).__init__()
    self.output_size = output_size
    self.dropout = dropout
    self.bert =  AutoModel.from_pretrained("vinai/phobert-base", num_labels=2, output_hidden_states=True)
    self.fc1 = nn.Linear(768, 768, bias=True)
    self.layernorm_1 = nn.LayerNorm((768,), eps = 1e-05, elementwise_affine=True)
    self.fc2 = nn.Linear(4*768, self.output_size, bias=True)
    nn.init.normal_(self.fc1.weight, std=0.02)
    nn.init.normal_(self.fc1.bias, 0)
    nn.init.normal_(self.fc2.weight, std=0.02)
    nn.init.normal_(self.fc2.bias, 0)

  def forward(self, inputs, attention_mask):
    outputs = self.bert(input_ids=inputs, attention_mask=attention_mask)
    x = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
#     x = F.dropout(x, self.dropout, inplace=False)
#     x = self.fc1(x)
#     x = self.layernorm_1(x)
#     x = F.dropout(x, self.dropout, inplace=False)
    x = self.fc2(x)
#     out = F.softmax(x, dim=-1)
    out = x
    return out


In [49]:
model = BERT_for_sentiment_large(output_size=2)

In [50]:
for i, child in enumerate(model.children()):
    print("i = ", i, " ", child)

i =  0   RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          

In [51]:
import torch.nn as nn
import torch.optim as optim
from transformers import get_linear_schedule_with_warmup

max_epochs = 10
learning_rate = 2*1e-5

weight_decay = 0.01
optimizing_parameters = filter(lambda p: p.requires_grad, model.parameters())
criterion = nn.CrossEntropyLoss()

optimizer = optim.AdamW(
    optimizing_parameters,
    lr=learning_rate,
    weight_decay=weight_decay
)
lr_scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=0,
                num_training_steps=len(train_dataloader)*max_epochs
            )

In [52]:
import numpy as np
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [53]:
device

device(type='cuda')

In [54]:
model = model.to(device)

In [55]:
from tqdm import tqdm, tqdm_notebook

In [57]:
def train_epoch(model,criterion, optimizer,scheduler, train_loader):
  total_loss = 0
  total_acc = 0
  total = 0
  tmp = 0
  model.train()

  for batch in tqdm(train_loader):
    input_ids, input_mask, labels = batch

    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    output = model(input_ids, attention_mask=input_mask)
    loss = criterion(output, labels)

    loss.backward()

    nn.utils.clip_grad_norm_(optimizing_parameters, max_norm=1.0)
    optimizer.step()
    scheduler.step()

    total_loss += loss.item()
    total += len(labels)

    logits = output.detach().cpu().numpy()
    labels = labels.to('cpu').numpy()
    acc = flat_accuracy(logits, labels)

    total_acc += acc

    tmp += 1

  return total_loss/total, total_acc / tmp

In [58]:
def validate_epoch(model,criterion,valid_loader):
    model.eval()
    total_loss = total = 0
    accuracy = 0
    eval_f1 = 0
    tmp = 0
    for batch in tqdm(valid_loader):
      input_ids, input_mask, labels = batch

      input_ids = input_ids.to(device=device)
      input_mask = input_mask.to(device=device)
      labels = labels.to(device=device)

          # Forwards pass
      with torch.no_grad():
        logits = model(input_ids, attention_mask=input_mask)

          # Calculate how wrong the model is
        loss = criterion(logits, labels)

        logits = logits.detach().cpu().numpy()
        labels = labels.to('cpu').numpy()


        acc = flat_accuracy(logits, labels)
        accuracy += acc
        # Record metrics
        total_loss += loss.item()
        total += len(labels)
        tmp += 1
    return total_loss / total , accuracy / tmp

In [64]:
max_epochs = 10
n_epochs = 0
train_losses, valid_losses = [], []
train_acces, valid_acces = [], []
for _ in range(max_epochs):
    train_loss, train_acc = train_epoch(model,criterion, optimizer,lr_scheduler, train_dataloader)
    valid_loss, valid_acc = validate_epoch(model,criterion, valid_dataloader)

    tqdm.write(
        f'epoch #{n_epochs + 1:3d}\n'
        f'train_loss: {train_loss:.2e}'
        f'\ttrain_acc: {train_acc:.2e}\n'
        f'valid_loss: {valid_loss:.2e}'
        f'\tvalid_acc: {valid_acc:.2e}\n',
    )

    print("saving model:...")
    torch.save(model.state_dict(), f'/phobert_linear_train_crawl_{n_epochs+1}.pt')
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    train_acces.append(train_acc)
    valid_acces.append(valid_acc)
    n_epochs += 1

100%|██████████| 57/57 [00:23<00:00,  2.40it/s]


epoch #  1
train_loss: 6.43e-03	train_acc: 9.30e-01
valid_loss: 6.76e-03	valid_acc: 9.24e-01

saving model:...


100%|██████████| 57/57 [00:23<00:00,  2.39it/s]


epoch #  2
train_loss: 5.66e-03	train_acc: 9.42e-01
valid_loss: 7.21e-03	valid_acc: 9.26e-01

saving model:...


100%|██████████| 57/57 [00:23<00:00,  2.40it/s]


...

epoch #  8
train_loss: 2.78e-03	train_acc: 9.76e-01
valid_loss: 8.41e-03	valid_acc: 9.25e-01

saving model:...


100%|██████████| 57/57 [00:23<00:00,  2.40it/s]


epoch #  9
train_loss: 2.50e-03	train_acc: 9.80e-01
valid_loss: 8.48e-03	valid_acc: 9.25e-01

saving model:...


100%|██████████| 57/57 [00:23<00:00,  2.39it/s]


epoch # 10
train_loss: 2.50e-03	train_acc: 9.78e-01
valid_loss: 8.47e-03	valid_acc: 9.25e-01

saving model:...


In [59]:
from tqdm import tqdm

In [60]:
test_model = BERT_for_sentiment_large()

In [65]:
test_model.load_state_dict(torch.load("/phobert_linear_train_crawl_3.pt"))

<All keys matched successfully>

In [91]:
test_data = pd.read_csv("/test_shuffle.csv")
test_data

,Comment,Rating
0,"Nước trong hình là blue sky , mình uống thì th...",1
1,"Mình cũng hay ăn bánh_xèo ở đây , bánh nhiều n...",0
2,Đây là trà xanh Táo & Nho_Mỹ : > Mỗi ngày 1 hư...,1
3,Phải nói đây là quán cơm hến mình ưng_ý từ trư...,0
4,Trong thời_tiết oi_bức ở sài gòn vậy_mà không ...,1
...,...,...
9995,"Bánh_mì được , ăn lạ_miệng , view đẹp . Tuy_nh...",0
9996,"Mình ko thích món bánh bạch_tuộc ở đây lắm , t...",0
9997,Nói_chung tất_cả các món ở hệ_thống món Huế ko...,0
9998,"Không_gian đẹp , nhưng mình thấy nước uống khô...",0


In [92]:
test_data = test_data.drop('Rating',axis = 1)
test_data['Comment']

0       Nước trong hình là blue sky , mình uống thì th...
1       Mình cũng hay ăn bánh_xèo ở đây , bánh nhiều n...
2       Đây là trà xanh Táo & Nho_Mỹ : > Mỗi ngày 1 hư...
3       Phải nói đây là quán cơm hến mình ưng_ý từ trư...
4       Trong thời_tiết oi_bức ở sài gòn vậy_mà không ...
                              ...                        
9995    Bánh_mì được , ăn lạ_miệng , view đẹp . Tuy_nh...
9996    Mình ko thích món bánh bạch_tuộc ở đây lắm , t...
9997    Nói_chung tất_cả các món ở hệ_thống món Huế ko...
9998    Không_gian đẹp , nhưng mình thấy nước uống khô...
9999    Mình thất_vọng về nước lẩu , thất_vọng ko thể ...
Name: Comment, Length: 10000, dtype: object

In [99]:
test_corpus = []
for sent in test_data["Comment"]:
  if(type(sent) != str):
    sent = "bình luận không xác định"
  test_corpus.append((id, sent))

print(test_corpus[0])

(<built-in function id>, 'Nước trong hình là blue sky , mình uống thì thấy không có gì đặc_trưng , bánh thì không ngon bằng Red_Velvet , phục_vụ ở đây tận_tình lắm . Ai đến đây thì cũng phải thử qua Red velvet vì nó quá ngon , muốn thay_đổi khẩu_vị nên gọi 2 em mới này mà thấy không ngon bằng . Không_gian ở Lumos phía ngoài thoáng mát lại đẹp nữa . Bên trong thì cũng bình thương\r\n')


In [100]:
sents = [sent for _,sent in test_corpus]


In [101]:
purified_sents = [purify(str(sent)) for sent in sents]

In [ ]:
purified_sents

In [103]:
preprocessed_sents = preprocess(purified_sents)

In [104]:
padded_sents, attention_masks = encode_plus(preprocessed_sents, max_len=256)

In [105]:
batch_size = 32

inputs = torch.tensor(padded_sents)
masks = torch.tensor(attention_masks)

test_data = TensorDataset(masks, inputs)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [106]:
inputs.shape

torch.Size([10000, 256])

In [ ]:
test_model.to(device)

In [108]:
sub = []
test_model.eval()
total_loss = total = 0
for batch in tqdm(test_dataloader):
# batch = torch.toTensor(t.to(device) for t in batch)
  input_mask, input_ids = batch

  input_ids = input_ids.to(device)
  input_mask = input_mask.to(device)

          # Forwards pass
  with torch.no_grad():
    outputs = test_model(input_ids, attention_mask=input_mask)
    # loss = outputs[0]
#     outputs = outputs.logits
    loss = outputs.detach().cpu().numpy()
    preds_flat = np.argmax(loss, axis=1).flatten()
    for tup in preds_flat:
      sub.append(tup)

100%|██████████| 313/313 [02:04<00:00,  2.52it/s]


In [110]:
len(sub)

10000

In [111]:
sub[0:]

[1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
